In [ ]:
from math import exp, log, sqrt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt

from regress import plot_fit

from linear_motion_sim import Simulator, fit_ks_and_kv, fit_ka, lognorm
from func_guesser import sign

In [ ]:
tgt_v = 2

In [ ]:
def s(t):
    return 20

def a(t):
    return 22

def d(t):
    return 0.8

sim = Simulator(max_speed_f=s, max_accel_f=a, drag_f=d, noise=0.2)

In [ ]:
class Speedometer:
    def __init__(self, num_samples):
        self.num_samples = num_samples
        self.t = []
        self.x = []


    def sample(self, t, x):
        while len(self.t) >= self.num_samples:
            del self.t[0]

        while len(self.x) >= self.num_samples:
            del self.x[0]

        self.t.append(t)
        self.x.append(x)

    def has_speed(self):
        return len(self.t) > 2


    def speed(self):
        if len(self.t) < 2:
            return 0
        
        return np.polyfit(self.t, self.x, 1)[0]

In [ ]:
class IntervalTimer:
    def __init__(self, interval):
        self.interval = interval
        self.last = -1e99


    def past_interval(self, t):
        if t - self.last > self.interval:
            self.last = t
            return True
        else:
            return False

In [ ]:
sped = Speedometer(5)
accel = Speedometer(5)
ival = IntervalTimer(0.05)

tlist, plist, xlist = [], [], []
# pest = ParamEstimator(5)

powr = 0.1

def motor_power(t, x):
    powr = 0.1 + t / 30

    tlist.append(t)
    plist.append(powr)
    xlist.append(x)
    return powr

In [ ]:
t, x, v, a, p  = sim.sim(motor_power)
vt = np.ones_like(t) * tgt_v

plt.figure(figsize=(20, 6))
plt.plot(t, v)
plt.plot(t, vt, '--')
plt.show()


In [ ]:
class ParamEstimator:
    def __init__(self, samp_window_size):
        assert samp_window_size % 2 == 1
        self.samp_window_size = samp_window_size
        self.t = []
        self.p = []
        self.x = []


    def sample(self, t, p, x):
        self.t.append(t)
        self.p.append(p)
        self.x.append(x)


    def estimate(self):
        assert len(self.t) > self.samp_window_size

In [ ]:
pest = ParamEstimator(5)
for i in range(800):
    pest.sample(tlist[i], plist[i], xlist[i])

In [ ]:
x = np.array([1, 1.5, 3])
y = 3*x**2 + 2.1*x+5
np.polyfit(x,y,2)

In [ ]:
def pow_vel_accel(t, p, x):
    i = t.size // 2
    a2, a1, _ = np.polyfit(t, x, 2)
    v = 2 * a2 * t[i] + a1
    a = 2 * a2
    return p[i], v, a

In [ ]:
self = pest
t = np.array(self.t)
x = np.array(self.x)
p = np.array(self.p)

m = (0.1 < p)&(p < 0.2)
t = t[m]
x = x[m]
p = p[m]

w = self.samp_window_size
n = t.size - w + 1

pva = np.empty((n, 3))
for i in range(n):
    pva[i] = pow_vel_accel(t[i:i + w], p[i:i + w], x[i:i + w])

In [ ]:
pow_vel_1 = np.empty((n, 3))
pow_vel_1 = pva.copy()
pow_vel_1[:, 2] = 1
accel = pva[:, 2]

In [ ]:
x, resid, rank, _ = np.linalg.lstsq(pow_vel_1, accel)
x

In [ ]:
df = pd.DataFrame({
    'p': pow_vel_1[:, 0],
    'v': pow_vel_1[:, 1],
    'a': accel
})
df.iloc[::5].to_clipboard()

In [ ]:
expect = sim.max_accel_f(0), -sim.max_accel_f(0) / sim.max_speed_f(0), -sim.drag_f(0)
expect